In [9]:
from tpot import TPOTRegressor
from sklearn.linear_model import LassoCV
import numpy as np
import pandas as pd

In [10]:
X = pd.read_pickle("cached_dataframe.pkl")

In [11]:
X = X.drop(columns=["material", "formula", "structure", "composition"])
X = X.loc[:, (X!=0).any(axis=0)]

In [ ]:
# Functionalizing features
functions = {
    "squared": lambda x: x**2,
    "cubed": lambda x: x**3
}

X_copy = X.copy()
for key,value in functions.items():
    tmp_df = pd.DataFrame()
    tmp_df = X_copy.applymap(value)
    tmp_df.columns = [f"{key}({col})" for col in tmp_df.columns]
    X = pd.concat([X, tmp_df], axis=1)
pd.set_option("display.max_columns", 400)
print(X)

   Li Intercalation Energy @ 0.25 Li/M  Volume Change @ 0.25 Li/M  \
0                               -4.193                  22.820453   
1                               -3.004                  21.404065   
2                               -1.605                   3.582239   
3                               -1.925                  11.386800   
4                               -2.635                   9.331461   
5                               -2.372                   4.140496   
6                               -4.741                  15.813220   

   Charge on Li @ 0.25 Li/M  Average Li-M Distance @ 0.25 Li/M  \
0                     0.850                              3.433   
1                     0.882                              3.055   
2                     0.830                              2.787   
3                     0.845                              2.821   
4                     0.850                              2.877   
5                     0.874                        

/tmp/ipykernel_843108/2878974050.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tmp_df = X_copy.applymap(value)
/tmp/ipykernel_843108/2878974050.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tmp_df = X_copy.applymap(value)


In [14]:
Y_arr = np.array([0.42, 0.94, 0.76, 0.37, 0.25, 0.91, 0.71])
X_arr = X.to_numpy()

In [17]:
# Lasso Regularization
lasso = LassoCV(cv=5, alphas=np.logspace(-6,6,100), random_state=10)
lasso.fit(X_arr, Y_arr)
print(lasso.alpha_)
print(lasso.coef_)

/home/sbhimineni/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.299e-03, tolerance: 3.092e-05
  model = cd_fast.enet_coordinate_descent(
/home/sbhimineni/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.764e-02, tolerance: 3.092e-05
  model = cd_fast.enet_coordinate_descent(
/home/sbhimineni/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

1000000.0
[-0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+0

/home/sbhimineni/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.755e-03, tolerance: 4.447e-05
  model = cd_fast.enet_coordinate_descent(
/home/sbhimineni/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.468e-03, tolerance: 4.447e-05
  model = cd_fast.enet_coordinate_descent(
/home/sbhimineni/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

In [ ]:
# Pearson Correlation
corr = X.corr(method="pearson")
print(corr)

In [ ]:
tpot = TPOTRegressor(generations=30, population_size=10, scoring='r2')
tpot.fit(X_arr,Y_arr)


TPOTRegressor(generations=30, population_size=10, scoring='r2')

In [66]:
tpot.export('best_ml_algo.py')